<a href="https://colab.research.google.com/github/JunHyeong-data/Basic-Deep-Learning/blob/main/03_backpropagation_%EC%97%AD%EC%A0%84%ED%8C%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝 기초 ③
## Backpropagation (백프로파게이션)

---

## 1. 강의 목표
이번 강의의 목표는 다음과 같다.
- Backpropagation의 실제 동작 원리 이해
- Gradient가 어떻게 계산되는지 수식으로 직접 확인
- PyTorch의 자동 미분이 무엇을 하는지 직관적으로 이해

이번 강의에서는 **“미분값을 마법처럼 얻는 과정”**을 직접 계산해본다.

---

## 2. 이전 내용 복습
이전 강의에서는 다음을 배웠다.
- Loss Function을 정의하여 모델의 성능을 수치화
- Gradient Descent를 사용해 Loss를 최소화
- 하지만 각 파라미터에 대한 미분값은 **직접 계산하지 않고 자동으로 얻음**

이번 강의의 핵심 질문은 다음이다.
> 🔹 이 미분값은 도대체 어떻게 계산되는가?

---

## 3. Backpropagation을 위한 최소한의 수학
Backpropagation을 이해하기 위해 필요한 수학은 다음뿐이다.

### (1) 기본 미분
$$f(x) = ax^3 + bx^2 + cx + d$$
$$\frac{df}{dx} = 3ax^2 + 2bx + c$$

### (2) Chain Rule (연쇄 법칙)
$$\frac{dL}{dx} = \frac{dL}{dz} \cdot \frac{dz}{dx}$$

👉 **이 두 가지만 알면 충분하다.**

---

## 4. 문제 단순화
이전에는 7개의 점을 사용했지만, 이번에는 문제를 극단적으로 단순화한다.

### 주어진 데이터
- 단 하나의 데이터 포인트: $(x, y) = (2, 3)$

### 모델
$$y = ax$$

### Loss Function
$$L = (ax - y)^2$$

👉 목표: **Loss를 최소화하는 $a$ 찾기**

---

## 5. 계산 그래프 (Computation Graph)
Loss 계산 과정은 다음과 같이 분해할 수 있다.

1. $u = ax$
2. $v = u - y$
3. $L = v^2$

이 구조를 **계산 그래프**라고 부른다. PyTorch도 내부적으로 이 그래프를 생성한다.



---

## 6. Forward Pass
초기값을 다음과 같이 설정한다.
- $a = 3, x = 2, y = 3$

계산 순서는 다음과 같다.
1. $u = ax = 3 \times 2 = 6$
2. $v = u - y = 6 - 3 = 3$
3. $L = v^2 = 3^2 = 9$

👉 **Forward Pass 결과: Loss = 9**

---

## 7. Backward Pass (Backpropagation)
이제 Loss에 대한 각 변수의 미분값을 계산한다.



### (1) $\frac{dL}{dL}$
$$\frac{dL}{dL} = 1$$

### (2) $\frac{dL}{dv}$
$$L = v^2 \Rightarrow \frac{dL}{dv} = 2v$$
$$v = 3 \Rightarrow \frac{dL}{dv} = 6$$

### (3) $\frac{dL}{du}$
$$v = u - y \Rightarrow \frac{dv}{du} = 1$$
$$\frac{dL}{du} = \frac{dL}{dv} \cdot \frac{dv}{du} = 6 \times 1 = 6$$

### (4) $\frac{dL}{da}$
$$u = ax \Rightarrow \frac{du}{da} = x$$
$$\frac{dL}{da} = \frac{dL}{du} \cdot \frac{du}{da} = 6 \times 2 = 12$$

### (5) $\frac{dL}{dx}$
$$\frac{dL}{dx} = \frac{dL}{du} \cdot a = 6 \times 3 = 18$$

### (6) $\frac{dL}{dy}$
$$\frac{dL}{dy} = \frac{dL}{dv} \cdot \frac{dv}{dy} = 6 \times (-1) = -6$$

---

## 8. Backpropagation 결과 요약

| 변수 | Gradient (기울기) |
| :--- | :--- |
| **$a$** | **12** |
| $x$ | 18 |
| $y$ | -6 |

👉 우리가 업데이트를 위해 원하는 값은 **$\frac{dL}{da} = 12$** 이다.

---

## 9. PyTorch로 동일한 계산 수행
PyTorch에서는 다음 과정을 자동으로 수행한다.
1. Forward Pass 중 계산 그래프 생성
2. `loss.backward()` 호출
3. Chain Rule을 이용해 모든 Gradient 계산

In [2]:
import torch

x = torch.tensor(2, dtype=torch.float32, requires_grad=True)
y = torch.tensor(3, dtype=torch.float32, requires_grad=True)

a = torch.tensor(3, dtype=torch.float32, requires_grad=True)

In [3]:
loss = (a*x - y)**2
print(f"loss {loss}")

loss.backward()

print(f"x grad : {x.grad}")
print(f"y grad : {y.grad}")
print(f"a grad : {a.grad}")
"""
뜻: $\frac{\partial L}{\partial a}$ (Loss를 $a$로 미분한 값)해석: 현재 상태에서 $a$가 1만큼 커지면 Loss는 a.grad만큼 변한다는 뜻
만약 a.grad가 12라면: $a$를 살짝 키우면 오차가 커지니, $a$를 줄여야 한다는 신호
만약 a.grad가 -5라면: $a$를 키우면 오차가 줄어드니, $a$를 더 키워야 한다는 신호
""

loss 9.0
x grad : 18.0
y grad : -6.0
a grad : 12.0


In [5]:
"""
a = torch.tensor(3, dtype=torch.float32, requires_grad=True)
losses = []
for idx in range(100):
    loss = (a*x - y)**2
    losses.append(loss.item())
    grad = loss.backward()
    print(f"a value: {a.item()}, dl/da :  {a.grad}" )
    with torch.no_grad():
        a -= 0.01 * a.grad

    a.grad.zero_()
"""

a = torch.tensor(3, dtype=torch.float32, requires_grad=True)
losses = []
for idx in range(100):
  loss = (a*x - y) ** 2
  losses.append(loss.item())
  grad = loss.backward()

  print(f"a value: {a.item()}, dl/da : {a.grad}, a.grad : {a.grad}")
  with torch.no_grad():
    a -= 0.01 * a.grad

  a.grad.zero_()

a value: 3.0, dl/da : 12.0, a.grad : 12.0
a value: 2.880000114440918, dl/da : 11.040000915527344, a.grad : 11.040000915527344
a value: 2.7696001529693604, dl/da : 10.156801223754883, a.grad : 10.156801223754883
a value: 2.668032169342041, dl/da : 9.344257354736328, a.grad : 9.344257354736328
a value: 2.574589490890503, dl/da : 8.596715927124023, a.grad : 8.596715927124023
a value: 2.4886224269866943, dl/da : 7.908979415893555, a.grad : 7.908979415893555
a value: 2.4095325469970703, dl/da : 7.2762603759765625, a.grad : 7.2762603759765625
a value: 2.3367700576782227, dl/da : 6.694160461425781, a.grad : 6.694160461425781
a value: 2.2698285579681396, dl/da : 6.158628463745117, a.grad : 6.158628463745117
a value: 2.208242177963257, dl/da : 5.665937423706055, a.grad : 5.665937423706055
a value: 2.151582717895508, dl/da : 5.2126617431640625, a.grad : 5.2126617431640625
a value: 2.0994560718536377, dl/da : 4.795648574829102, a.grad : 4.795648574829102
a value: 2.051499605178833, dl/da : 4.4119